## **Assessing Healthcare Accessibility: A Timor-Leste Case Study**

### This Jupyter notebook aims to evaluate the percentage of the population with access to hospitals and clinics in Timor-Leste, utilizing Python and geospatial analysis. It combines geospatial data from various sources, such as GADM, Facebook population data, and OpenStreetMap, to determine healthcare access within specific administrative regions. The notebook involves spatial visualization, spatial joins, and isochrone analysis to estimate the catchment areas of healthcare facilities. Finally, it calculates the percentage of the population with healthcare access and visualizes the results on an interactive map using Folium.

* Importing Required Libraries: The notebook begins by installing and importing the necessary Python libraries, including Folium, pandas, geopandas, and others.

* Loading Administrative Boundary Data: It uses the GADM library to obtain geospatial data for Timor-Leste's administrative boundaries, specifically at the first administrative level (e.g., districts or provinces). The data is then visualized on a map.

* Facebook Population Data: Facebook population data for Timor-Leste in 2020 is retrieved and processed, creating a geospatial dataset of population distribution.

* Spatial Join: A spatial join operation is performed to find the population within the selected administrative region (e.g., "Baucau").

* Healthcare Facility Data: Healthcare facility data, including hospitals and clinics, is collected from OpenStreetMap using Overpass API. The data is cleaned and prepared for analysis.

* Catchment Area Analysis: Isochrone analysis is conducted to estimate the catchment areas of healthcare facilities. This analysis helps determine which population groups have access to healthcare services from each facility.

* Population with Access Calculation: The notebook calculates the percentage of the population with access to healthcare services in the selected administrative region.

* Interactive Map Visualization: The results are visualized on an interactive map using Folium, displaying healthcare facilities, population with and without access, and administrative boundaries.

The notebook concludes by summarizing the findings and the overall percentage of the population with healthcare access in the selected region.




In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import pickle, hashlib, os 
from functools import wraps

def disk_cache(cache_dir="cache"):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Ensure the cache directory exists
            os.makedirs(cache_dir, exist_ok=True)

            # Create a hash key from the function name and arguments
            hash_key = hashlib.sha256()
            hash_key.update(func.__name__.encode())
            hash_key.update(pickle.dumps(args))
            hash_key.update(pickle.dumps(kwargs))
            filename = f"{cache_dir}/{hash_key.hexdigest()}.pkl"

            # Check if the cache file exists
            if os.path.exists(filename):
                with open(filename, 'rb') as f:
                    return pickle.load(f)
            else:
                # Call the function and cache its result
                result = func(*args, **kwargs)
                with open(filename, 'wb') as f:
                    pickle.dump(result, f)
                return result
        return wrapper
    return decorator

In [ ]:
if IN_COLAB:
    %pip install gadm hdx-python-api
    %pip install geopandas --upgrade
    %pip install pyomo
    %pip install highspy
    %pip install chart_studio

In [ ]:
import folium as fl
import pandas as pd
import geopandas as gpd
from hdx.api.configuration import Configuration
from hdx.data.resource import Resource
import urllib.request
import requests
import json
import requests
import itertools

from shapely.geometry import Polygon,MultiPolygon
from shapely.ops import unary_union

from gadm import GADMDownloader
import numpy as np

import pyomo.environ as pyo

import plotly.express as px

**GADM Data**

### GADM, which stands for "Global Administrative Areas," is a valuable resource in the field of geospatial analysis and cartography. GADM provides comprehensive and up-to-date geographical data on administrative boundaries for countries worldwide. Here are some key points to note about GADM data:

1. **Administrative Boundary Data:** GADM offers geospatial data representing administrative divisions within countries. This includes boundaries for countries, states, provinces, districts, municipalities, and other administrative units. Users can access boundary data at various administrative levels.

2. **Global Coverage:** GADM covers nearly every country on Earth, making it a truly global resource. This extensive coverage is invaluable for researchers, analysts, and cartographers working with geospatial information.

3. **Accurate and Detailed:** GADM data is known for its accuracy and detail. It is curated and maintained to provide the most up-to-date and reliable information on administrative boundaries. This level of precision is essential for various applications, including geographic analysis and map creation.

4. **Open Access:** GADM data is typically available as open data, meaning it can be freely accessed, downloaded, and used by the public. This open access policy encourages widespread use and collaboration in the geospatial community.

5. **Diverse Use Cases:** GADM data finds applications in a wide range of fields, including urban planning, epidemiology, environmental science, and social sciences. It supports research, decision-making, and policy development by providing a foundation for spatial analysis.

6. **Spatial Analysis:** Researchers and analysts often use GADM data for tasks such as spatial joins, geospatial modeling, demographic analysis, and accessibility assessments. It forms the basis for answering questions related to administrative boundaries and geographic distribution.

7. **Map Visualization:** Cartographers use GADM data to create maps at various scales and levels of detail. These maps are instrumental in illustrating administrative divisions, electoral boundaries, and other geographic features.

8. **Consistent Format:** GADM data is typically available in standard geospatial file formats such as Shapefiles or GeoJSON, making it compatible with a wide range of Geographic Information System (GIS) software and tools.

In summary, GADM data is a valuable resource for accessing high-quality, up-to-date geographical information on administrative boundaries worldwide. Its open access nature and precision make it an essential asset for professionals and researchers working in the field of geospatial analysis and mapping.

In [ ]:
# Initialize the GADMDownloader with the specified version (in this case, version 4.0)
downloader = GADMDownloader(version="4.0")

# Define the country name for which you want to retrieve administrative boundary data
country_name = "Timor-Leste"

# Specify the administrative level you are interested in (e.g., 1 for districts or provinces)
ad_level = 1

# Retrieve the geospatial data for the selected country and administrative level
gdf = downloader.get_shape_data_by_country_name(country_name=country_name, ad_level=ad_level)

# Display the first 2 rows of the obtained geospatial data for a quick preview
gdf.head(2)


In [ ]:
# Create a Folium map (m) with an initial zoom level of 10 and using OpenStreetMap tiles as the basemap
m = fl.Map(location=[-8.556856, 125.560314], zoom_start=9, tiles="OpenStreetMap")

# Iterate through each row in the geospatial data (gdf) representing administrative boundaries
for _, r in gdf.iterrows():
    # Simplify the geometry of the current boundary with a specified tolerance
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.00001)

    # Convert the simplified geometry to JSON format
    geo_j = sim_geo.to_json()

    # Create a GeoJson layer from the JSON geometry, and style it with an orange fill color
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "orange"})

    # Add a popup with the NAME_1 attribute (administrative region name) to the GeoJson layer
    #fl.Popup(r["NAME_1"]).add_to(geo_j)

    # Add the styled GeoJson layer to the Folium map (m)
    geo_j.add_to(m)

# Display the Folium map (m) with the administrative boundaries and popups
m


In [ ]:
selected_adm1 = 'Baucau'

In [ ]:
selected_gadm = gdf[gdf['NAME_1']==selected_adm1]

for _, r in selected_gadm.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = fl.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "red"})
    fl.Popup(r["NAME_1"]).add_to(geo_j)
    geo_j.add_to(m)
m

**High Resolution Population Density Maps**

### Accurate population density data plays a crucial role in delivering social services effectively. Meta, in collaboration with the Center for International Earth Science Information Network (CIESIN), has developed highly accurate population maps that offer valuable insights into population distribution. Key highlights of these population density maps include:

1. **High Resolution:** These maps provide population estimates at a high resolution of 30 meters, allowing for detailed analysis of population distribution.

2. **Demographic Breakdowns:** In addition to total population density, the maps offer demographic breakdowns, including data on women, men, youth, children, women of reproductive age, and the elderly, all at the same 30-meter resolution.

3. **Public Availability:** These population density maps are publicly accessible and cover over 160 countries and territories worldwide. They can be downloaded from platforms such as Humanitarian Data Exchange (HDX) and Amazon Web Services (AWS).

The process of creating these maps involves several key steps:

- **Step 1: Model Population Growth:** CIESIN records census data and uses it to model population growth at both country and subnational levels.

- **Step 2: Satellite Imagery Analysis:** Computer algorithms are trained to analyze satellite imagery. The presence of buildings and structures is used as an indicator of human population.

- **Step 3: Building Density Calculation:** Algorithms calculate the density of buildings within each 30x30 meter tile in the satellite image.

- **Step 4: Population Estimation:** Population data is distributed across the tiles based on building density, resulting in actionable population density maps.

These maps serve various purposes, including urban planning, public health, disaster response, and environmental studies. They are available for download through HDX for desktop use in GIS software or through AWS Public Datasets for large-scale programmatic usage via Amazon S3 and Athena.

These population density maps represent a significant resource for researchers, policymakers, and organizations working to better understand and address population-related challenges across the globe.

In [ ]:
def fb_pop_data(country_iso3: str) -> pd.DataFrame:
    """
    Get 2020 facebook data for an area defined by the MultiPolygon geometry
    """
    try:
        Configuration.create(
            hdx_site="prod", user_agent="Get_Population_Data", hdx_read_only=True
        )
    except:
        pass
    resource = Resource.search_in_hdx(
        f"name:{country_iso3.lower()}_general_2020_csv.zip"
    )
    url = resource[0]["download_url"]
    filehandle, _ = urllib.request.urlretrieve(url)
    print("Data downloaded")
    facebook_pop_csv = pd.read_csv(filehandle, compression="zip")
    population = facebook_pop_csv.reset_index()
    population.columns = ['ID','xcoord','ycoord','population']

    population_meta = gpd.GeoDataFrame(population,
                                       geometry=gpd.points_from_xy(x=population.xcoord,
                                                                   y=population.ycoord))

    return population_meta

population_meta = fb_pop_data('TLS')
population_meta = population_meta.set_crs(selected_gadm.crs)
print('Total Population:',round(population_meta['population'].sum()/1000000,2),'million')

In [ ]:
population_meta.head(2)

In [ ]:
# Perform a spatial join to find population within the selected administrative boundary
population_aoi = gpd.sjoin(population_meta, selected_gadm)
print('Total Population (Area of Interest -', selected_adm1,'):',round(population_aoi['population'].sum()))

### This code segment retrieves and analyzes healthcare facility data (hospitals and clinics) in Timor-Leste within a specified area of interest (AOI). Here's a brief summary of what it does:

- It uses the Overpass API to query OpenStreetMap data for hospitals in Timor-Leste, retrieves the data in JSON format, and converts it into a DataFrame (`df_hospitals`).

- It extracts relevant information, such as the hospital's name, latitude, and longitude, from the OpenStreetMap data.

- Similarly, it queries OpenStreetMap data for clinics in Timor-Leste, retrieves the data, and processes it into a DataFrame (`df_clinics`), extracting relevant information.

- The code then combines the hospital and clinic data into a single GeoDataFrame (`df_health_osm`) and converts latitude and longitude coordinates into a geometry column.

- It prints the number of hospitals and clinics extracted from the data.

- Finally, it performs a spatial join to determine how many hospitals and clinics fall within the specified administrative region of interest (AOI) and prints the result.

This code segment is a critical step in assessing healthcare accessibility in a specific region of Timor-Leste, as it identifies and quantifies the healthcare facilities within the chosen area.

In [ ]:
%%time

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="TL"];
(node["amenity"="hospital"](area);
 way["amenity"="hospital"](area);
 rel["amenity"="hospital"](area);
);
out center;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
data = response.json()

df_hospitals = pd.DataFrame(data['elements'])

df_hospitals['name'] = df_hospitals['tags'].apply(lambda x:x['name'] if 'name' in list(x.keys()) else None)

df_hospitals = df_hospitals[['id','lat','lon','name']].drop_duplicates()

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area["ISO3166-1"="TL"];
(node["amenity"="clinic"](area);
 way["amenity"="clinic"](area);
 rel["amenity"="clinic"](area);
);
out center;
"""
response = requests.get(overpass_url,
                        params={'data': overpass_query})
data = response.json()

df_clinics = pd.DataFrame(data['elements'])
df_clinics['name'] = df_clinics['tags'].apply(lambda x:x['name'] if 'name' in list(x.keys()) else None)
df_clinics['amenity'] = df_clinics['tags'].apply(lambda x:x['healthcare'] if 'healthcare' in list(x.keys()) else None)

df_clinics = df_clinics[['id','lat','lon','name','amenity']].drop_duplicates()

df_health_osm = pd.concat([df_hospitals,df_clinics])
df_health_osm = gpd.GeoDataFrame(df_health_osm, geometry=gpd.points_from_xy(df_health_osm.lon, df_health_osm.lat))
df_health_osm = df_health_osm[['id','name','geometry']]

print('Number of hospitals and clinics extracted:',len(df_health_osm))
df_health_osm = df_health_osm.set_crs(selected_gadm.crs)
selected_hosp = gpd.sjoin(df_health_osm, selected_gadm, predicate='within')
print('Number of hospitals and clinics in AOI (',selected_adm1,'):',len(selected_hosp))

### This code defines a function, `get_isochrone_osm`, which calculates an isochrone polygon representing a reachable area around a given location (usually a hospital). It makes an API call to OpenRouteService to obtain a polygon based on a specified travel time (here, 1800 seconds or 30 minutes). The resulting polygon is returned as a geometric shape.

In [ ]:
len(selected_hosp)

In [ ]:
@disk_cache('cache/osm')
def get_isochrone_osm (each_hosp,travel_time_secs):
  body = {"locations":[[each_hosp.x,each_hosp.y]],"range":[travel_time_secs],"range_type":'time'}
  headers = {
      'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
      'Authorization': 'OVERPASS API KEY',
      'Content-Type': 'application/json; charset=utf-8'
  }
  call = requests.post('https://api.openrouteservice.org/v2/isochrones/foot-walking', json=body, headers=headers)
  if(call.status_code==200):
    geom = (json.loads(call.text)['features'][0]['geometry'])
    polygon_geom = Polygon(geom['coordinates'][0])
    return polygon_geom
  else:
    return None

In [ ]:
@disk_cache('cache/mapbox')
def get_isochrone_mapbox (each_hosp,minutes,access_token,mode):
  longitude = each_hosp.x
  latitude = each_hosp.y
  query = """https://api.mapbox.com/isochrone/v1/mapbox/"""
  query = query+mode+'/'
  query = query+str(longitude)+','+str(latitude)+'?'
  query = query+'contours_minutes='+minutes
  query = query+'&polygons=true&access_token='
  query = query+access_token
  req_return = (requests.get(query).json())

  if('code' in req_return):
    if (req_return['code']=='NoSegment'):
      print('No Segment')
    else:
      print(req_return)
  else:
    return(req_return['features'])

In [ ]:
selected_hosp['cachment_area_osm'] = selected_hosp['geometry'].apply(get_isochrone_osm,travel_time_secs=3600)

In [ ]:
quartile_labels = [0.1, 0.25, 0.5, 1.0]
population_aoi['opacity'] = pd.qcut(population_aoi['population'], 4, labels=quartile_labels)

In [ ]:
population_aoi

In [ ]:
jg_mapbox_key = list(pd.read_csv('JG_mapbox.csv',sep=';').to_dict().keys())[0]

In [ ]:
access_token = jg_mapbox_key#'MAPBOX ACCESS KEY SHOULD BE ADDED HERE!'

In [ ]:
selected_hosp['cachment_area_mapbox'] = selected_hosp['geometry'].apply(get_isochrone_mapbox,minutes="60",access_token=access_token,mode='walking')
selected_hosp['cachment_area_mapbox'] = selected_hosp['cachment_area_mapbox'].apply(lambda x: x[0]['geometry'])
selected_hosp['cachment_area_mapbox'] = selected_hosp['cachment_area_mapbox'].apply(lambda x:Polygon(x['coordinates'][0]))

In [ ]:
selected_hosp['cachment_area'] = selected_hosp['cachment_area_mapbox']

In [ ]:
def get_pop_count(cachment,pop_data):
  if(cachment!=None):
    pop_access = pop_data[pop_data.within(cachment)]
    id_values = (pop_access['ID'].values)
    pop_with_access = (pop_access['population'].sum().round())
    return id_values,pop_with_access
  else:
    return [None,None]

selected_hosp['id_with_access'], selected_hosp['pop_with_access'] = zip(*selected_hosp['cachment_area'].apply(get_pop_count, pop_data=population_aoi))


In [ ]:
list_ids_access = list(selected_hosp['id_with_access'].values)
list_ids_access = list(itertools.chain.from_iterable(list_ids_access))
pop_with_access = population_aoi[population_aoi['ID'].isin(list_ids_access)]
pop_without_access = population_aoi[~population_aoi['ID'].isin(list_ids_access)]

original_access = round(pop_with_access['population'].sum()*100/population_aoi['population'].sum(),2)

print('Population with Access:',round(pop_with_access['population'].sum()*100/population_aoi['population'].sum(),2),'%')

In [ ]:
view_one = selected_hosp[selected_hosp['id']==11641171758]

folium_map = fl.Map(location=[-8.475, 126.456], zoom_start=11, tiles="OpenStreetMap")

geo_adm = fl.GeoJson(data=selected_gadm.iloc[0]['geometry'],style_function=lambda x:{'color': 'orange'})
geo_adm.add_to(folium_map)

for i in range(0,len(view_one)):
    fl.Marker([view_one.iloc[i]['geometry'].y, view_one.iloc[i]['geometry'].x],
                        color='blue',popup=view_one.iloc[i]['name']).add_to(folium_map)

geo_polygon = fl.GeoJson(data=view_one.iloc[0]['cachment_area_mapbox'],style_function=lambda x:{'color': 'green'})
geo_polygon.add_to(folium_map)

folium_map


In [ ]:
pop_access_one = population_aoi[population_aoi['ID'].isin(view_one['id_with_access'].values[0])]

for i in range(0,len(pop_access_one)):
  fl.CircleMarker(
        location=[pop_access_one.iloc[i]['ycoord'], pop_access_one.iloc[i]['xcoord']],
        radius=5,
        color=None,
        fill=True,
        fill_color='green',
        fill_opacity=pop_without_access.iloc[i]['opacity']).add_to(folium_map)

folium_map

In [ ]:
folium_map = fl.Map(location=[-8.475, 126.456], zoom_start=11, tiles="OpenStreetMap")

geo_adm = fl.GeoJson(data=selected_gadm.iloc[0]['geometry'],style_function=lambda x:{'color': 'orange'})
geo_adm.add_to(folium_map)

for i in range(0,len(selected_hosp)):
    fl.Marker([selected_hosp.iloc[i]['geometry'].y, selected_hosp.iloc[i]['geometry'].x],
                        color='blue',popup=selected_hosp.iloc[i]['name']).add_to(folium_map)

for i in range(0,len(pop_without_access)):
  fl.CircleMarker(
        location=[pop_without_access.iloc[i]['ycoord'], pop_without_access.iloc[i]['xcoord']],
        radius=5,
        color=None,
        fill=True,
        fill_color='red',
        fill_opacity=pop_without_access.iloc[i]['opacity']).add_to(folium_map)

for i in range(0,len(pop_with_access)):
  fl.CircleMarker(
        location=[pop_with_access.iloc[i]['ycoord'], pop_with_access.iloc[i]['xcoord']],
        radius=5,
        color=None,
        fill=True,
        fill_color='green',
        fill_opacity=pop_without_access.iloc[i]['opacity']).add_to(folium_map)

folium_map


In [ ]:
def generate_grid_in_polygon(
    spacing: float, geometry: MultiPolygon
) -> gpd.GeoDataFrame:
    """
    This Function generates evenly spaced points within the given GeoDataFrame.
    The parameter 'spacing' defines the distance between the points in coordinate units.
    """

    # Get the bounds of the polygon
    minx, miny, maxx, maxy = geometry.bounds

    # Square around the country with the min, max polygon bounds
    # Now generate the entire grid
    x_coords = list(np.arange(np.floor(minx), int(np.ceil(maxx)), spacing))
    y_coords = list(np.arange(np.floor(miny), int(np.ceil(maxy)), spacing))
    mesh = np.meshgrid(x_coords, y_coords)
    grid = gpd.GeoDataFrame(
        data={"longitude": mesh[0].flatten(), "latitude": mesh[1].flatten()},
        geometry=gpd.points_from_xy(mesh[0].flatten(), mesh[1].flatten()),
        crs="EPSG:4326",
    )
    grid = gpd.clip(grid, geometry)
    grid = grid.reset_index(drop=True).reset_index().rename(columns={"index": "ID"})

    return grid

In [ ]:
potential_locations = generate_grid_in_polygon(geometry=selected_gadm['geometry'].values[0],spacing=0.02)
len(potential_locations)

In [ ]:
folium_map = fl.Map([-8.475, 126.456], zoom_start=11)
geo_adm = fl.GeoJson(data=selected_gadm.iloc[0]['geometry'],style_function=lambda x:{'color': 'orange'})

for i in range(0,len(potential_locations)):
  fl.CircleMarker(
        location=[potential_locations.iloc[i]['latitude'], potential_locations.iloc[i]['longitude']],
        radius=3,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(folium_map)

folium_map

In [ ]:
potential_locations['cachment_area_mapbox'] = potential_locations['geometry'].apply(get_isochrone_mapbox,minutes="60",access_token=access_token,mode='walking')
potential_locations['cachment_area_mapbox'] = potential_locations['cachment_area_mapbox'].apply(lambda x: x[0]['geometry'])
potential_locations['cachment_area_mapbox'] = potential_locations['cachment_area_mapbox'].apply(lambda x:Polygon(x['coordinates'][0]))

In [ ]:
potential_locations['id_with_access'], potential_locations['pop_with_access'] = zip(*potential_locations['cachment_area_mapbox'].apply(get_pop_count, pop_data=population_aoi))

In [ ]:
potential_locations

In [ ]:
list_ids_access_old = list(selected_hosp['id_with_access'].values)
list_ids_access_old = list(itertools.chain.from_iterable(list_ids_access_old))

list_ids_access_new = list(potential_locations['id_with_access'].values)
list_ids_access_new = list(itertools.chain.from_iterable(list_ids_access_new))

list_ids_access = list_ids_access_old + list_ids_access_new

pop_with_access = population_aoi[population_aoi['ID'].isin(list_ids_access)]
pop_without_access = population_aoi[~population_aoi['ID'].isin(list_ids_access)]

print('Maximum access attainable with this potential location list:',round(pop_with_access['population'].sum()*100/population_aoi['population'].sum(),2),'%')
max_access_possible = round(pop_with_access['population'].sum()*100/population_aoi['population'].sum(),2)

In [ ]:
def model_max_covering(w, I, J, JI, p, J_existing):

    assert set(J_existing).issubset(set(J))

    m = pyo.ConcreteModel('MaxCovering')

    m.p = pyo.Param(mutable=True, within=pyo.Integers, default=p)
    m.I = pyo.Set(initialize=I)
    m.J = pyo.Set(initialize=J)
    m.Jfixed = pyo.Set(initialize=J_existing)
    m.nof_fixed = pyo.Param(mutable=False, within=pyo.Integers, default=len(J_existing))

    @m.Param(m.I, within=pyo.NonNegativeReals)
    def w(m, i):
        return w[i]

    @m.Param(m.I, within=pyo.Any)
    def JI(m, i):
        return JI.get(i,[])

    m.x = pyo.Var(m.J, within=pyo.Binary)
    m.z = pyo.Var(m.I, within=pyo.Binary)

    @m.Objective(sense=pyo.maximize)
    def covering(m):
        return pyo.quicksum(m.w[i] * m.z[i] for i in m.I)

    @m.Constraint(m.I)
    def serve_if_reachable_and_open(m, i):
        return m.z[i] <= pyo.quicksum(m.x[j] for j in m.JI[i])

    @m.Constraint()
    def budget(m):
        return pyo.quicksum(m.x[j] for j in m.J) <= m.nof_fixed + m.p

    @m.Constraint(m.Jfixed)
    def fix_open(m,j):
        return m.x[j] == 1

    return m

def get_selected(variables):
    return [k for k, v in variables.items() if v() > 0.5]

In [ ]:
w = population_aoi.set_index('ID').population.to_dict()

J_existing = set(selected_hosp.id)
J_potential = set(potential_locations.ID )

J = sorted( J_existing | J_potential )
I = sorted( set(population_aoi.ID) )

IJ_existing = selected_hosp.set_index('id').id_with_access.to_dict()
IJ_potential = potential_locations.set_index('ID').id_with_access.to_dict()

IJ = IJ_existing | IJ_potential

def reverse_mapping( mapping ):
    from collections import defaultdict
    aux = defaultdict(set)
    for x, Y in mapping.items():
        for y in Y:
            aux[y].add(x)
    return { y : sorted(aux[y]) for y in sorted(aux.keys()) }

JI = reverse_mapping( IJ )
J_existing = sorted(J_existing)

In [ ]:
model = model_max_covering(w, I, J, JI, 0, J_existing)
solver = pyo.SolverFactory('appsi_highs')

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
list_ids_access_old = list(selected_hosp['id_with_access'].values)
list_ids_access_old = list(itertools.chain.from_iterable(list_ids_access_old))
result_list = []
for each_val in tqdm(range(0,len(potential_locations))):
  model.p = each_val
  solver.solve(model)
  opened_ids = get_selected(model.x)
  selected_new = potential_locations[potential_locations['ID'].isin(opened_ids)]
  list_ids_access_new = list(selected_new['id_with_access'].values)
  list_ids_access_new = list(itertools.chain.from_iterable(list_ids_access_new))
  list_ids_access = list_ids_access_old + list_ids_access_new
  pop_with_access = population_aoi[population_aoi['ID'].isin(list_ids_access)]
  pop_without_access = population_aoi[~population_aoi['ID'].isin(list_ids_access)]
  pop_percentage = round(pop_with_access['population'].sum()*100/population_aoi['population'].sum(),2)
  result_list.append([each_val+len(selected_hosp),pop_percentage])

In [ ]:
# Extract data for plotting
x_values = [item[0] for item in result_list]
y_values = [item[1] for item in result_list]

# Create a plotly graph
fig = px.scatter(x=x_values, y=y_values)

# Update the layout
fig.update_layout(
    xaxis_title="Number of facilities (existing + new)",
    yaxis_title="Percentage of population with access",
    plot_bgcolor='white',
    yaxis=dict(range=[0, 100]),
    xaxis=dict(range=[0, 100]),
    width=1200
)

# Add vertical line and annotation at x=14
fig.add_vline(x=14, line_width=3, line_dash="dash", line_color="green")
fig.add_annotation(
    x=14, y=50,
    text="Number of existing facilities",
    showarrow=True,
    arrowhead=1,
    ax=20,
    ay=-30
)

# Add vertical line and annotation at x=14
fig.add_hline(y=max_access_possible, line_width=3, line_dash="dash", line_color="green")
fig.add_annotation(
    x=13, y=87,
    text="Maximum access possible with the potential location list",
    showarrow=True,
    arrowhead=1,
    ax=10,
    ay=-30
)

# Show the figure
fig.show()

In [ ]:
model.p = 3
solver.solve(model)
opened_ids = get_selected(model.x)
selected_new = potential_locations[potential_locations['ID'].isin(opened_ids)]
list_ids_access_old = list(selected_hosp['id_with_access'].values)
list_ids_access_old = list(itertools.chain.from_iterable(list_ids_access_old))
list_ids_access_new = list(selected_new['id_with_access'].values)
list_ids_access_new = list(itertools.chain.from_iterable(list_ids_access_new))
list_ids_access = list_ids_access_old + list_ids_access_new
pop_with_access = population_aoi[population_aoi['ID'].isin(list_ids_access)]
pop_without_access = population_aoi[~population_aoi['ID'].isin(list_ids_access)]


In [ ]:
folium_map = fl.Map(location=[-8.475, 126.456], zoom_start=11, tiles="OpenStreetMap")

geo_adm = fl.GeoJson(data=selected_gadm.iloc[0]['geometry'],style_function=lambda x:{'color': 'orange'})
geo_adm.add_to(folium_map)

for i in range(0,len(selected_hosp)):
    fl.Marker([selected_hosp.iloc[i]['geometry'].y, selected_hosp.iloc[i]['geometry'].x],
                        color='blue',popup=selected_hosp.iloc[i]['name']).add_to(folium_map)

for i in range(0,len(selected_new)):
    fl.Marker([selected_new.iloc[i]['geometry'].y, selected_new.iloc[i]['geometry'].x],
                        icon=fl.Icon(color='darkpurple')).add_to(folium_map)

for i in range(0,len(pop_without_access)):
  fl.CircleMarker(
        location=[pop_without_access.iloc[i]['ycoord'], pop_without_access.iloc[i]['xcoord']],
        radius=5,
        color=None,
        fill=True,
        fill_color='red',
        fill_opacity=pop_without_access.iloc[i]['opacity']).add_to(folium_map)

for i in range(0,len(pop_with_access)):
  fl.CircleMarker(
        location=[pop_with_access.iloc[i]['ycoord'], pop_with_access.iloc[i]['xcoord']],
        radius=5,
        color=None,
        fill=True,
        fill_color='green',
        fill_opacity=pop_with_access.iloc[i]['opacity']).add_to(folium_map)

folium_map